# Laboratorio: Modelos del lenguaje con RNNs (Word2Vec)

Aarón Medina Melián

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Portada_Trafalgar_%281873%29.jpg/800px-Portada_Trafalgar_%281873%29.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano de Trafalgar, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro Trafalgar, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

In [2]:
## TU CÓDIGO AQUÍ
path = keras.utils.get_file(fname="trafalgar.txt", origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ")

Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [3]:
## TU CÓDIGO AQUÍ
text = open(path, encoding="utf8").read().lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado

In [4]:
## TU CÓDIGO AQUÍ
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 300039
-i-

se me permitirá que antes de referir el gran suceso de que fui testigo,
diga algunas palabras sobre mi infancia, explicando por qué extraña
manera me llevaron los azares de la vida a presenciar la terrible
catástrofe de nuestra marina.

al hablar de mi nacimiento, no imitaré a la mayor parte de


In [5]:
text = text[5:len(text)]

text

'se me permitirá que antes de referir el gran suceso de que fui testigo,\ndiga algunas palabras sobre mi infancia, explicando por qué extraña\nmanera me llevaron los azares de la vida a presenciar la terrible\ncatástrofe de nuestra marina.\n\nal hablar de mi nacimiento, no imitaré a la mayor parte de los que\ncuentan hechos de su propia vida, quienes empiezan nombrando su\nparentela, las más veces noble, siempre hidalga por lo menos, si no se\ndicen descendientes del mismo emperador de trapisonda. yo, en esta\nparte, no puedo adornar mi libro con sonoros apellidos; y fuera de mi\nmadre, a quien conocí por poco tiempo, no tengo noticia de ninguno de\nmis ascendientes, si no es de adán, cuyo parentesco me parece\nindiscutible. doy principio, pues, a mi historia como pablos, el buscón\nde segovia: afortunadamente dios ha querido que en esto sólo nos\nparezcamos.\n\nyo nací en cádiz, y en el famoso barrio de la viña, que no es hoy, ni\nmenos era entonces, academia de buenas costumbres. la 

In [6]:
fin = "fin de trafalgar"
posicion = text.find(fin)


print(len(fin))
print(posicion)

text[posicion:len(text)]

16
299920


'fin de trafalgar\n\nmadrid, enero-febrero 1873.\n\nbenito pérez galdós; edición ilustrada por enrique y arturo mélida\n'

In [7]:
text = text[0:posicion+len(fin)]

text

'se me permitirá que antes de referir el gran suceso de que fui testigo,\ndiga algunas palabras sobre mi infancia, explicando por qué extraña\nmanera me llevaron los azares de la vida a presenciar la terrible\ncatástrofe de nuestra marina.\n\nal hablar de mi nacimiento, no imitaré a la mayor parte de los que\ncuentan hechos de su propia vida, quienes empiezan nombrando su\nparentela, las más veces noble, siempre hidalga por lo menos, si no se\ndicen descendientes del mismo emperador de trapisonda. yo, en esta\nparte, no puedo adornar mi libro con sonoros apellidos; y fuera de mi\nmadre, a quien conocí por poco tiempo, no tengo noticia de ninguno de\nmis ascendientes, si no es de adán, cuyo parentesco me parece\nindiscutible. doy principio, pues, a mi historia como pablos, el buscón\nde segovia: afortunadamente dios ha querido que en esto sólo nos\nparezcamos.\n\nyo nací en cádiz, y en el famoso barrio de la viña, que no es hoy, ni\nmenos era entonces, academia de buenas costumbres. la 

## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*Se me permitirá que antes de referir el gr*" -> predicción: **a**
* "*e me permitirá que antes de referir el gra*" -> predicción: **n**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: e me permitirá que antes de referir el gra

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: a

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante (PERO USANDO PALABRAS NO CARACTERES).

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de PALABRAS con la siguiente PALABRA a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* PALABRAS (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de las palabras y mapas de palabras

Antes que nada, necesitamos saber qué palabras aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_words* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_words* - 1. Por ejemplo, {'se': 0, 'me': 1, ...}
3.   Diccionario reverso de índices a palabras: {0: 'se', 1: 'me', ...}


In [8]:
import re
from unidecode import unidecode

In [9]:
def quitar_tildes_y_mantener_n(texto):
    # Reemplazar letras acentuadas con sus equivalentes sin acento
    texto_sin_tildes = ''
    for caracter in texto:
        if caracter == 'ñ' or caracter == 'Ñ':
            texto_sin_tildes += caracter
        else:
            texto_sin_tildes += unidecode(caracter)
    return texto_sin_tildes

In [10]:
text = re.sub(r'[^A-Za-záéíóúüÁÉÍÓÚÜñÑ\s]', '', text)
# Sustituir letras con tilde por su versión sin tilde
text = quitar_tildes_y_mantener_n(text)

In [11]:
print(text)

se me permitira que antes de referir el gran suceso de que fui testigo
diga algunas palabras sobre mi infancia explicando por que extraña
manera me llevaron los azares de la vida a presenciar la terrible
catastrofe de nuestra marina

al hablar de mi nacimiento no imitare a la mayor parte de los que
cuentan hechos de su propia vida quienes empiezan nombrando su
parentela las mas veces noble siempre hidalga por lo menos si no se
dicen descendientes del mismo emperador de trapisonda yo en esta
parte no puedo adornar mi libro con sonoros apellidos y fuera de mi
madre a quien conoci por poco tiempo no tengo noticia de ninguno de
mis ascendientes si no es de adan cuyo parentesco me parece
indiscutible doy principio pues a mi historia como pablos el buscon
de segovia afortunadamente dios ha querido que en esto solo nos
parezcamos

yo naci en cadiz y en el famoso barrio de la viña que no es hoy ni
menos era entonces academia de buenas costumbres la memoria no me da
luz alguna sobre mi persona 

In [12]:
palabras = text.split()
palabras_unicas = set(palabras)
word_index = {}
k = 0
for palabra in palabras_unicas:
    word_index[palabra] = k
    k+=1

In [13]:
reversed_word_index = dict([(value, key) for (key, value) in word_index.items()])

#### 2.2. Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y las correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente palabra a predecir. Una vez hecho, desplazarse una palabra a la derecha y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y las palabras a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 2, tendríamos

* *sequences* = ["Don Quijote", "Quijote de"]
* *next_chars* = ['de', 'La']

In [14]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 5
step = 1
sequences = []
next_words = []

for i in range(0,len(palabras)-SEQ_LENGTH, step):
  sequences.append(palabras[i:i+SEQ_LENGTH])
  next_words.append(palabras[i+SEQ_LENGTH])

In [15]:
print('Ejemplo: secuencia número 5:')
print(sequences[4])
print('Siguiente caracter:')
print(next_words[4])

Ejemplo: secuencia número 5:
['antes', 'de', 'referir', 'el', 'gran']
Siguiente caracter:
suceso


Indicar el tamaño del training set que acabamos de generar.

In [16]:
## TU CÓDIGO AQUÍ
print('número de datos de training: {}'.format(len(sequences)))

número de datos de training: 51039


Como el libro es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [17]:
MAX_SEQUENCES = 50000

perm = np.random.permutation(len(sequences))
sequences, next_words = np.array(sequences), np.array(next_words)
sequences, next_words = sequences[perm], next_words[perm]
sequences, next_words = list(sequences[:MAX_SEQUENCES]), list(next_words[:MAX_SEQUENCES])

print(len(sequences))

50000


In [18]:
sequences[len(sequences)-1]

array(['viejo', 'no', 'desplego', 'sus', 'labios'], dtype='<U21')

In [19]:
next_words[len(sequences)-1]

'antes'

In [20]:
import nltk
nltk.download('punkt')

frases = []

for i in range(len(sequences)):
    frase = ' '.join(sequences[i])
    frases.append(frase)
newsVec = [nltk.word_tokenize(frase) for frase in frases]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aaron\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
newsVec

[['accidentes', 'muy', 'visibles', 'en', 'su'],
 ['tenia', 'cierta', 'cultura', 'o', 'delicadeza'],
 ['los', 'heridos', 'que', 'quedaban', 'en'],
 ['vil', 'estado', 'esto', 'no', 'pasa'],
 ['cabeza', 'se', 'erizan', 'todavia', 'al'],
 ['puñetazo', 'en', 'la', 'mesa', 'si'],
 ['jesus', 'si', 'yo', 'gastara', 'calzones'],
 ['soñar', 'todavia', 'me', 'incorpore', 'en'],
 ['se', 'crecia', 'ella', 'mas', 'ante'],
 ['indignacion', 'y', 'los', 'dolores', 'de'],
 ['estaba', 'en', 'todas', 'las', 'cabezas'],
 ['en', 'los', 'barquitos', 'digo', 'que'],
 ['viaje', 'a', 'la', 'martinica', 'y'],
 ['los', 'sentidos', 'con', 'la', 'contemplacion'],
 ['que', 'iba', 'a', 'callar', 'por'],
 ['agujeros', 'hechos', 'en', 'el', 'casco'],
 ['su', 'bendita', 'mujer', 'restame', 'hablar'],
 ['gente', 'de', 'poco', 'mas', 'o'],
 ['faltaba', 'el', 'ultimo', 'resto', 'del'],
 ['desde', 'el', 'introito', 'hasta', 'el'],
 ['en', 'ellos', 'este', 'se', 'nego'],
 ['aventurado', 'plan', 'de', 'nelson', 'que'],
 ['bie

In [22]:
from gensim.models import Word2Vec

modelWord2Vec = Word2Vec(newsVec, min_count = 1, vector_size = 32)

In [23]:
modelWord2Vec.wv["orgullosa"]

array([-0.05513502, -0.25252995,  0.198192  ,  0.23960291,  0.02030977,
       -0.20065075,  0.26062447,  0.21827978, -0.02191248, -0.05814039,
        0.16049092, -0.15674902,  0.06345789, -0.08751602, -0.08988661,
        0.00083971, -0.1215039 ,  0.09427904,  0.00984074,  0.16350913,
        0.25192827,  0.32418865,  0.3885202 , -0.12324031,  0.07503459,
        0.02402988, -0.20109119,  0.10773151,  0.02536635, -0.15613846,
       -0.1660402 ,  0.05602018], dtype=float32)

In [24]:
modelWord2Vec.wv.most_similar("orgullosa")

[('dirigio', 0.9937586784362793),
 ('conmigo', 0.9932805299758911),
 ('furia', 0.993046224117279),
 ('mucha', 0.9929311275482178),
 ('derecho', 0.9928731918334961),
 ('haberse', 0.9928702116012573),
 ('continuo', 0.992857038974762),
 ('tales', 0.9928374290466309),
 ('profundo', 0.9928065538406372),
 ('gabriel', 0.9923704862594604)]

#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestras palabras. Por ejemplo, si sólo tuviéramos 4 palabras (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_words)* e **y** tendrá shape *(num_sequences, num_words)*.



In [25]:
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, 32))
y = np.zeros((NUM_SEQUENCES, len(palabras_unicas)))

In [26]:
for k in range(NUM_SEQUENCES):
    palabra = next_words[k]
    y[k, word_index[palabra]] = 1 #codificación ONE-HOT

In [27]:
for k in range(NUM_SEQUENCES):
    for w in range(SEQ_LENGTH):
        word = sequences[k][w]
        X[k, w] = modelWord2Vec.wv[word]

## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [28]:
## TU CÓDIGO AQUÍ
vocabulary_size = len(palabras_unicas)

In [29]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, 32)))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 8487)              1094823   
                                                                 
Total params: 1177255 (4.49 MB)
Trainable params: 1177255 (4.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(X, y, validation_split=0.5, batch_size=128, epochs=1, shuffle=True).history

196/196 [==============================] - 11s 47ms/step - loss: 7.1688 - accuracy: 0.0484 - val_loss: 6.9148 - val_accuracy: 0.0496


In [31]:
X[3]

array([[-0.01956171, -0.06087212,  0.08553395,  0.12114449,  0.03589514,
        -0.03733461,  0.05739801,  0.07235398, -0.01033412, -0.01525344,
         0.0765212 , -0.04162123,  0.00849302, -0.0374366 , -0.06265275,
         0.00371085, -0.0224467 ,  0.03410124, -0.04362135,  0.07240555,
         0.12239145,  0.14496134,  0.13466507, -0.03107755,  0.03999886,
         0.05792977, -0.07357457,  0.029534  ,  0.01950829, -0.10327251,
        -0.0879354 ,  0.00773017],
       [-0.19592677, -0.65313202,  0.6154536 ,  0.65646124,  0.02023046,
        -0.42164639,  0.71688771,  0.5529027 , -0.04888514, -0.09009815,
         0.44438756, -0.35438988,  0.31636095, -0.22098516, -0.20947221,
         0.03348077, -0.26843792,  0.19955362,  0.04385242,  0.46554714,
         0.66859019,  0.97939771,  1.1729387 , -0.37442449,  0.08001772,
         0.22980072, -0.57920158,  0.05242782,  0.09959083, -0.49126014,
        -0.32811651,  0.12358955],
       [-0.35044947, -0.50898945,  0.87984061,  1.0812

In [32]:
probs = model.predict(X)

1563/1563 [==============================] - 6s 3ms/step


In [33]:
print(probs[1])

[5.5134231e-05 9.1196462e-06 4.3761320e-05 ... 2.7220922e-06 2.3182963e-06
 3.2332326e-05]


Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [34]:
## TU CÓDIGO AQUÍ
def sample(probs, temperature=1.0):
    probs = np.asarray(probs).astype('float64')
    
    probs = np.log(probs) / temperature
    
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)

Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [35]:
## TU CÓDIGO AQUÍ
TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 10

In [36]:
def generate_text(seed_text, model, length, temperature=1):
    generated = seed_text
    
    texto = seed_text.split()
    
    NUM_TEXTO = len(texto)

    X_pred = np.zeros((NUM_TEXTO, SEQ_LENGTH, 32))
    for k in range(length):
        for a in range(NUM_TEXTO):
            for w in range(5):
                word = texto[a]
                X[a, w] = modelWord2Vec.wv[word]
        prediccion = model.predict(X_pred, batch_size = 32, verbose = 0)
        generated += " " + reversed_word_index[sample(prediccion[0], temperature)]
                
    return generated

In [37]:
def on_epoch_end(epoch, logs):
  print("\n\n\n")

  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(palabras) - SEQ_LENGTH)
  seed_text = " ".join(palabras[start_pos:start_pos + SEQ_LENGTH])
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))

    generated_text = generate_text(seed_text, model,
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))
    print()


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [38]:
## TU CÓDIGO AQUÍ
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=10,shuffle=True,verbose=0,callbacks=[generation_callback]).history





------> Epoch: 1 - Generando texto con temperature 0.2
Seed: dios compre algunas golosinas mas
Texto generado: dios compre algunas golosinas mas de y a de y de en y y de

------> Epoch: 1 - Generando texto con temperature 0.5
Seed: dios compre algunas golosinas mas
Texto generado: dios compre algunas golosinas mas un para despues tan recinto bordo era todos de me

------> Epoch: 1 - Generando texto con temperature 1.0
Seed: dios compre algunas golosinas mas
Texto generado: dios compre algunas golosinas mas cosas cayo efecto quedo metiendo bondadosos dijimos luchaban levantarse estrafalario

------> Epoch: 1 - Generando texto con temperature 1.2
Seed: dios compre algunas golosinas mas
Texto generado: dios compre algunas golosinas mas mentir aventurera maniobra barcos llamarse a causado preteritas refirieron borde





------> Epoch: 2 - Generando texto con temperature 0.2
Seed: de mi historia pues hoy
Texto generado: de mi historia pues hoy y y de de de de de que que que

------> Ep

## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2 usando El Quijote:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


